# Denoising Demo Notebook

Notebook to showcase some denoising methods for electron microscopy images


This notebook is intended to be used in Google Colab. <br>
If it is used locally all the necessary python packages need to be installed seperataly.

## How to use this notebook?

This notebook contains four image denoising methods. Before each method you need to declare some inputs, like the path to the directory containing your images and some method specific parameters. More information on each method can be found by following the provided links.
<br>
<br>
All methods should be used with .tif images only

# 0. Before getting started

## Imports

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from tifffile import imread, imsave, imwrite
from scipy import ndimage
import os
import random
import warnings
warnings.filterwarnings("ignore")

# 1. Median Filter

(source: https://en.wikipedia.org/wiki/Median_filter) <br>
(documentation: https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.median_filter.html)

## Inputs

In [ ]:
# Specify the input directory containing your source images
input_directory = 'images'

### Parameter

In [ ]:
# specify the size of the kernel used for denoising:
# the bigger the kernel, the more denoising & blurring
kernel_size=30

## Denoising function

In [ ]:
# Create the output directory if it doesn't exist
median_output_directory = 'median_results'
if not os.path.exists(median_output_directory):
    os.makedirs(median_output_directory)

# create empty list where names of all processed files will be saved
median_file_list=[]

# Loop through all files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith(".tif"):
        # Construct the full path for the input image
        input_path = os.path.join(input_directory, filename)

        # Read the image
        img = imread(input_path)

        # Apply the median filter
        filtered = ndimage.median_filter(img, size=kernel_size)

        # Construct the full path for the output image
        output_path = os.path.join(median_output_directory, filename)

        # Save the filtered image
        imwrite(output_path, filtered)

        # add to list
        median_file_list.append([input_path, output_path])
        
        print(f"Processed: {filename} and saved to {output_path}")

## Display a random pair of noisy and denoised images

In [ ]:
# get ramdom pair of original und processed images
random_pair=random.choice(median_file_list)
orig_img=imread(random_pair[0])
median_img=imread(random_pair[1])

# display images
f=plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.imshow(orig_img, interpolation='nearest', cmap='gray')
plt.title('Original')
plt.axis('off');
plt.subplot(1,2,2)
plt.imshow(median_img, interpolation='nearest', cmap='gray')
plt.title('Median filter')
plt.axis('off');
plt.subplots_adjust(wspace=0.05)

# 2. Anisotropic Diffusion

(source: https://en.wikipedia.org/wiki/Anisotropic_diffusion) <br>
(original code: https://pastebin.com/sBsPX4Y7)

## Inputs

In [ ]:
# Specify the input directory containing your source images
input_directory = 'images'

### Parameter

In [ ]:
# Number of Iterations
niter = 100

step=(1.,1.)
gamma=0.1
kappa=50

## Denoising function

In [ ]:
# Create the output directory if it doesn't exist
anisodif_output_directory = 'anisodif_results'
if not os.path.exists(anisodif_output_directory):
    os.makedirs(anisodif_output_directory)

# create empty list where names of all processed files will be saved
anisodif_file_list=[]

# Loop through all files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith(".tif"):
        # initialize output array
        img = img.astype('float32')
        imgout = img.copy()
 
        # initialize some internal variables
        deltaS = np.zeros_like(imgout)
        deltaE = deltaS.copy()
        NS = deltaS.copy()
        EW = deltaS.copy()
        gS = np.ones_like(imgout)
        gE = gS.copy()

        for ii in range(niter):
         
            # calculate the diffs
            deltaS[:-1,: ] = np.diff(imgout,axis=0)
            deltaE[: ,:-1] = np.diff(imgout,axis=1)
            gS = 1./(1.+(deltaS/kappa)**2.)/step[0]
            gE = 1./(1.+(deltaE/kappa)**2.)/step[1]
         
            # update matrices
            E = gE*deltaE
            S = gS*deltaS
         
            # subtract a copy that has been shifted 'North/West' by one
            # pixel. don't as questions. just do it. trust me.
            NS[:] = S
            EW[:] = E
            NS[1:,:] -= S[:-1,:]
            EW[:,1:] -= E[:,:-1]
         
            # update the image
            imgout += gamma*(NS+EW)
        
            iterstring = "Iteration %i" %(ii+1)

        # Construct the full path for the output image
        output_path = os.path.join(anisodif_output_directory, filename)
        
        # Save the filtered image
        imwrite(output_path, imgout)

        # add to list
        anisodif_file_list.append([input_path, output_path])
        
        print(f"Processed: {filename} and saved to {output_path}")

## Display a random pair of noisy and denoised images

In [ ]:
# get ramdom pair of original und processed images
random_pair=random.choice(anisodif_file_list)
orig_img=imread(random_pair[0])
median_img=imread(random_pair[1])

# display images
f=plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.imshow(orig_img, interpolation='nearest', cmap='gray')
plt.title('Original')
plt.axis('off');
plt.subplot(1,2,2)
plt.imshow(median_img, interpolation='nearest', cmap='gray')
plt.title('Anisotropic Diffusion, ' + iterstring)
plt.axis('off');
plt.subplots_adjust(wspace=0.05)

# 3. Non-local Means
(source: https://en.wikipedia.org/wiki/Non-local_means) <br>
(documentation: https://scikit-image.org/docs/stable/api/skimage.restoration.html#skimage.restoration.denoise_nl_means)

## Inputs

In [ ]:
# Specify the input directory containing your source images
input_directory = 'images'

### Parameter

In [ ]:
# size of the patches used for denoising
size_of_patches = 7

# Maximal distance in pixels where to search patches used for denoising
distance_to_patch = 11

# Cut-off distance (in gray levels). The higher h, the more permissive one is in accepting patches. 
# A higher h results in a smoother image, at the expense of blurring features. 
cut_off_dist=0.8

# The standard deviation of the (Gaussian) noise. 
# We use the estimate_sigma()-function since this is not known
# If you want to change this, set sigma here:
#sigma_value = 
# Dont forget to change the function in the following cell!

In [ ]:
# Create the output directory if it doesn't exist
nlmeans_output_directory = 'nlmeans_results'
if not os.path.exists(nlmeans_output_directory):
    os.makedirs(nlmeans_output_directory)

# create empty list where names of all processed files will be saved
nlmeans_file_list=[]

# Loop through all files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith(".tif"):
        # Construct the full path for the input image
        input_path = os.path.join(input_directory, filename)

        # Read the image
        img = imread(input_path)
        
        # Add a # at beginning of the following line of you want to provide a value for sigma
        sigma_value = np.mean(estimate_sigma(img))
        
        denoise_img = denoise_nl_means(img, 
                                       h=cut_off_dist*sigma_est,
                                       sigma=sigma_value,
                                       patch_size=size_of_patches,
                                       patch_distance=distance_to_patch)

        # Construct the full path for the output image
        output_path = os.path.join(nlmeans_output_directory, filename)
        
        # Save the filtered image
        imwrite(output_path, imgout)

        # add to list
        nlmeans_file_list.append([input_path, output_path])
        
        print(f"Processed: {filename} and saved to {output_path}")


## Display a random pair of noisy and denoised images

In [ ]:
# get ramdom pair of original und processed images
random_pair=random.choice(nlmeans_file_list)
orig_img=imread(random_pair[0])
median_img=imread(random_pair[1])

# display images
f=plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.imshow(orig_img, interpolation='nearest', cmap='gray')
plt.title('Original')
plt.axis('off');
plt.subplot(1,2,2)
plt.imshow(median_img, interpolation='nearest', cmap='gray')
plt.title('Non-local means')
plt.axis('off');
plt.subplots_adjust(wspace=0.05)

In [ ]:
test_img=imread('images/123.tif')
sigma_est=np.mean(estimate_sigma(test_img))
denoise_img = denoise_nl_means(test_img, h=cut_off_dist*sigma_est, sigma=sigma_est, patch_size=size_of_patches, patch_distance=distance_to_patch)



In [ ]:
# display images
f=plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.imshow(test_img, interpolation='nearest', cmap='gray')
plt.title('Original')
plt.axis('off');
plt.subplot(1,2,2)
plt.imshow(denoise_img, interpolation='nearest', cmap='gray')
plt.title('Anisotropic Diffusion, ')
plt.axis('off');
plt.subplots_adjust(wspace=0.05)

# 4. StructN2V
(source: https://github.com/juglab/n2v/blob/main/examples/2D/structN2V_2D_synth_mem/train_and_predict.ipynb) <br>
(documentation: https://github.com/juglab/n2v/tree/main)

## Imports
The imports for N2V are seperated from the rest because the installation of these is a little more complex. <br>
In case you want to run this notebook locally. <br>
The imports at the beginning of this notebook are necessary to use Noise2Void.

In [ ]:
import tensorflow as tf

# We import all our dependencies.
from csbdeep.io import save_tiff_imagej_compatible
import csv
from n2v.internals.N2V_DataGenerator import N2V_DataGenerator
from n2v.models import N2VConfig, N2V
from n2v.utils.n2v_utils import manipulate_val_data
import pandas as pd
import shutil
import time

In [ ]:
if tf.test.gpu_device_name()=='':
  print('You do not have GPU access.')
  print('Did you change your runtime ?')
  print('If the runtime setting is correct then Google did not allocate a GPU for your session')
  print('Expect slow performance. To access GPU try reconnecting later')

else:
  print('You have GPU access')
  !nvidia-smi

print('TensorFlow version:')
print(tf.__version__)

## Inputs

In [ ]:
#Path to where the training images are stored
training_source = 'n2v_training_images'

# Specifiy how the model should be namend and where it should be saved
model_name = "n2v_pop_test_uno"
model_path = "n2v_models"
full_model_path = os.path.join(model_path, model_name)

# create folder
if not os.path.exists(model_path):
    os.makedirs(model_path)

# Path to where the results should be saved
input_directory = 'images'
n2v_result_folder = 'N2V_results'

# create folder
if not os.path.exists(n2v_result_folder):
    os.makedirs(n2v_result_folder)

## Parameter

In [ ]:
#Number of epochs: default: 100
number_of_epochs =  10

#Patch size (pixels): default:64
patch_size =  64

#Batch size: default: 128
batch_size=64

#Fraction of training image set used for validation: default: 10
percentage_validation=10

#Initial learning Rate: default 0.0004
initial_learning_rate=0.0004

# Should data augmentation be used to diversify the training image data?
Use_Data_augmentation = False

# Do you want to use Noise2Void or structuredN2V
structured = True

# If you want to use StructuredN2V, enter the blinding mask here in the same fashion as the example 5x5:
blinding_mask = [[0,0,1,0,0],
                 [0,0,1,0,0],
                 [1,1,1,1,1],
                 [0,0,1,0,0],
                 [0,0,1,0,0]]


if structured:
    mask=blinding_mask
else:
    mask=None 

## Training Data Preperation

In [ ]:
# create DataGenerator-object.
datagen = N2V_DataGenerator()

#compatibility to easily change the name of the parameters
training_images = training_source
imgs = datagen.load_imgs_from_directory(directory = training_source)

In [ ]:
# split patches from the training images
Xdata = datagen.generate_patches_from_list(imgs, shape=(patch_size,patch_size), augment=Use_Data_augmentation)

shape_of_Xdata = Xdata.shape

# create a threshold (10 % patches for the validation)
threshold = int(shape_of_Xdata[0]*(percentage_validation/100))
# split the patches into training patches and validation patches
X = Xdata[threshold:]
X_val = Xdata[:threshold]
print(Xdata.shape[0],"patches created.")
print(threshold,"patch images for validation (",percentage_validation,"%).")
print(Xdata.shape[0]-threshold,"patch images for training.")

#Here we automatically define number_of_step in function of training data and batch size
#number_of_steps= int(X.shape[0]/batch_size)+1
number_of_steps= int(X.shape[0]/batch_size)+1


## Model configuration

In [ ]:
# create a Config object
config = N2VConfig(X, unet_kern_size=3,
                   train_steps_per_epoch=number_of_steps, train_epochs=number_of_epochs,
                   train_loss='mse', batch_norm=True, train_batch_size=batch_size, n2v_perc_pix=0.198,
                   n2v_manipulator='uniform_withCP', n2v_neighborhood_radius=5, train_learning_rate = initial_learning_rate,
                  structN2Vmask=mask)

# Let's look at the parameters stored in the config-object.
vars(config)

# create network model.
model = N2V(config=config, name=model_name, basedir=model_path)

## Training

### There will be an empty AssertionError: at the end of the training. You can ignore this. 

In [ ]:
start = time.time()

history = model.train(X, X_val)
print("Training, done.")

# convert the history.history dict to a pandas DataFrame:
lossData = pd.DataFrame(history.history)

if os.path.exists(os.path.join(full_model_path, "Quality Control")):
  shutil.rmtree(os.path.join(full_model_path, "Quality Control"))

os.makedirs(os.path.join(full_model_path, "Quality Control"))

# The training evaluation.csv is saved (overwrites the Files if needed).
lossDataCSVpath = os.path.join(full_model_path, "Quality Control", "training_evaluation.csv")
with open(lossDataCSVpath, 'w') as f:
  writer = csv.writer(f)
  writer.writerow(['loss','val_loss', 'learning rate'])
  for i in range(len(history.history['loss'])):
    writer.writerow([history.history['loss'][i], history.history['val_loss'][i], history.history['lr'][i]])

# Displaying the time elapsed for training
dt = time.time() - start
mins, sec = divmod(dt, 60)
hour, mins = divmod(mins, 60)
print("Time elapsed:",hour, "hour(s)",mins,"min(s)",round(sec),"sec(s)")

model.export_TF(name='Noise2Void',
                description='Noise2Void',
                authors=["Lucas Fortune"],
                test_img=X_val[0,...,0], axes='YX',
                patch_shape=(patch_size, patch_size))

## Quality control

In [ ]:
lossDataFromCSV = []
vallossDataFromCSV = []

with open(os.path.join(full_model_path, "Quality Control", "training_evaluation.csv"), "r") as csvfile:
    csvRead = csv.reader(csvfile, delimiter=',')
    next(csvRead)
    for row in csvRead:
        lossDataFromCSV.append(float(row[0]))
        vallossDataFromCSV.append(float(row[1]))

epochNumber = range(len(lossDataFromCSV))
plt.figure(figsize=(15,10))

plt.subplot(2,1,1)
plt.plot(epochNumber,lossDataFromCSV, label='Training loss')
plt.plot(epochNumber,vallossDataFromCSV, label='Validation loss')
plt.title('Training loss and validation loss vs. epoch number (linear scale)')
plt.ylabel('Loss')
plt.xlabel('Epoch number')
plt.legend()

plt.subplot(2,1,2)
plt.semilogy(epochNumber,lossDataFromCSV, label='Training loss')
plt.semilogy(epochNumber,vallossDataFromCSV, label='Validation loss')
plt.title('Training loss and validation loss vs. epoch number (log scale)')
plt.ylabel('Loss')
plt.xlabel('Epoch number')
plt.legend()
plt.savefig(os.path.join(full_model_path, 'Quality Control/lossCurvePlots.png'))
plt.show()

## Using the model

In [ ]:
n2v_file_list = []

#Activate the pretrained model.
config = None
model = N2V(config, model_name, basedir=model_path)

# Loop through the files
for r, d, f in os.walk(input_directory):
    for file in f:
        base_filename = os.path.basename(file)
        input_train = imread(os.path.join(r, file))
        pred_train = model.predict(input_train, axes='YX', n_tiles=(2,1))
        save_tiff_imagej_compatible(os.path.join(n2v_result_folder, base_filename), pred_train, axes='YX')

        # add to list
        in_path = input_directory+'/'+file
        out_path = n2v_result_folder+'/'+file
        n2v_file_list.append([in_path, out_path])
        
        print(f"Processed: {base_filename} and saved to {n2v_result_folder}")

## Display a random pair of noisy and denoised images

In [ ]:
# get ramdom pair of original und processed images
random_pair=random.choice(n2v_file_list)
orig_img=imread(random_pair[0])
median_img=imread(random_pair[1])

# display images
f=plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.imshow(orig_img, interpolation='nearest', cmap='gray')
plt.title('Original')
plt.axis('off');
plt.subplot(1,2,2)
plt.imshow(median_img, interpolation='nearest', cmap='gray')
plt.title('Noise2Void')
plt.axis('off');
plt.subplots_adjust(wspace=0.05)